# Convert corpus to non-zero state coefficients for quantum encoding

In [1]:
# Set the installation path of the intel-qnlp directory
QNLP_PATH="/Users/mlxd/workspace/quantum/intel-qnlp/"
corpus_name="jack_and_jill.txt"

In [2]:
# Load the module files assuming they are not on path
import importlib.util
spec = importlib.util.spec_from_file_location("tagging", QNLP_PATH+"py/tagging.py")
tg = importlib.util.module_from_spec(spec)
spec.loader.exec_module(tg)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mlxd/workspace/quantum/intel-qnlp/py/tagging.py'

In [3]:
import nltk
import sys
import numpy as np

from collections import Counter
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [4]:
"""
Remove words that do not add to the meaning; simplifies sentences
"""
def remove_stopwords(text, sw):
    return [w for w in text if w not in sw]

In [5]:
"""
Pass the corpus as a string, which is subsequently broken into tokenised sentences, and returned as dictionary.
proc_mode defines whether to enable lemmatization (proc_mode="l") or stemming (proc_mode="s"). Stop words can 
also be removed by setting stop_words=False
"""
def tokenize_corpus(corpus, proc_mode=0, stop_words=True):
    token_sents = nltk.sent_tokenize(corpus_text) #Split on sentences
    token_words = [] # Individual words
    tags = [] # Words and respective tags

    for s in token_sents:
        tk = nltk.word_tokenize(s)
        if stop_words == False:
            tk = remove_stopwords(tk, sw)
        token_words.extend(tk)
        tags.extend(nltk.pos_tag(tk))

    if proc_mode != 0:
        if proc_mode == 's':
            s = nltk.SnowballStemmer('english', ignore_stopwords=True)
            token_words = [s.stem(t) for t in token_words]
        elif proc_mode == 'l':
            wnl = nltk.WordNetLemmatizer()
            token_words = [wnl.lemmatize(t) for t in token_words]

    tags = nltk.pos_tag(token_words)

    nouns = [i[0] for i in tags if tg.matchables(tg.Noun, i[1])]
    verbs = [i[0] for i in tags if tg.matchables(tg.Verb, i[1])]

    count_nouns = Counter(nouns)
    count_verbs = Counter(verbs)
    return {'verbs':count_verbs, 'nouns':count_nouns, 'tk_sentence':token_sents, 'tk_words':token_words}

In [6]:
# Load the corpus
corpus_text=""

with open(QNLP_PATH+"corpus/"+corpus_name, 'r') as corpusFile:
    corpus_text=corpusFile.read()

words = tokenize_corpus(corpus_text, proc_mode=0)

#Naively set sentence boundary at location of full-stops.
sentence_boundary = set([i for i, x in enumerate(words['tk_words']) if "." in x])

In [7]:
"""
Examine the words around each verb with the specified window size, and attempt to match the NVN pattern.
The window_size specifies the number of values around each verb to search for the matching nouns. If passed as tuple
(l,r) gives the left and right windows specarately. If passed as a scalar, both values are equal.
"""
def word_pairing(words, window_size):
    if type(window_size) is tuple:
        window_left, window_right = window_size
    else:
        window_left = window_size
        window_right = window_size
    verb_idx={}
    
    for v in words['verbs']:
        verb_idx.update({v:[i for i, x in enumerate(words['tk_words']) if v in x]})
    
    for n in words['nouns']:
        for v,k in verb_idx.items():
            for v_idx in k:
                #Ensure that no full stop appears over the windowed region to avoid crossing sentences.
                l_left = len(sentence_boundary.intersection(range(v_idx-window_left,v_idx)))
                l_right = len(sentence_boundary.intersection(range(v_idx,v_idx+window_right)))
                
                if l_left==0 and n in words['tk_words'][v_idx-window_left:v_idx]:
                    print("LEFT:=",n,v)
                if l_right==0 and (v_idx + window_right) < len(words['tk_words']) and n in words['tk_words'][v_idx:v_idx+window_right] :
                    print("RIGHT:=",v,n)
    

In [8]:
#window = (2,3)
window=1
word_pairing(words, window)

LEFT:= Jack fell
LEFT:= Jack got
LEFT:= Jill went
LEFT:= Jill came
LEFT:= home did


In [9]:
"""
Calculate the required number of qubits to encode the NVN state-space of the corpus.
Returns tuple of required states and number of needed qubits. 
While it is possible to reduce qubit count by considering the total required number as being the product of each combo, 
it is easier to deal with separate states for nouns and verbs, with at most 1 extra qubit required.
"""
def num_qubits(words):
    nvn_space_size = len(words['nouns'])**2 * len(words['verbs'])
    req_qubits_n = int(np.ceil(np.log2(len(words['nouns']))))
    req_qubits_v = int(np.ceil(np.log2(len(words['verbs']))))
    
    #req_qubits = int(np.ceil(np.log2(nvn_space_size)))
    req_qubits = req_qubits_n*2 + req_qubits_v
    
    print("Unique states:",nvn_space_size,"\tRequired qubits total:", req_qubits, \
          "\tRequired qubits nouns:", req_qubits_n, "\tRequired qubits verbs:", req_qubits_v)
    return (nvn_space_size, req_qubits, req_qubits_n, req_qubits_v)

In [10]:
_, num_q_total, num_q_n, num_q_v = num_qubits(words)

Unique states: 1584 	Required qubits total: 12 	Required qubits nouns: 4 	Required qubits verbs: 4


In [11]:
#The following pairings represent the intended meaning space. Many of these are non-sensicle combinations, and so
#the resulting data set will be sparse, with nonzero values for observed appearances from the windowed operation.
for i in words['nouns']:
    for j in words['verbs']:
        for k in words['nouns']:
            print(i," ", j, " ",k )

Jack   went   Jack
Jack   went   Jill
Jack   went   hill
Jack   went   pail
Jack   went   water
Jack   went   crown
Jack   went   Up
Jack   went   home
Jack   went   As
Jack   went   head
Jack   went   vinegar
Jack   went   paper
Jack   fetch   Jack
Jack   fetch   Jill
Jack   fetch   hill
Jack   fetch   pail
Jack   fetch   water
Jack   fetch   crown
Jack   fetch   Up
Jack   fetch   home
Jack   fetch   As
Jack   fetch   head
Jack   fetch   vinegar
Jack   fetch   paper
Jack   fell   Jack
Jack   fell   Jill
Jack   fell   hill
Jack   fell   pail
Jack   fell   water
Jack   fell   crown
Jack   fell   Up
Jack   fell   home
Jack   fell   As
Jack   fell   head
Jack   fell   vinegar
Jack   fell   paper
Jack   broke   Jack
Jack   broke   Jill
Jack   broke   hill
Jack   broke   pail
Jack   broke   water
Jack   broke   crown
Jack   broke   Up
Jack   broke   home
Jack   broke   As
Jack   broke   head
Jack   broke   vinegar
Jack   broke   paper
Jack   came   Jack
Jack   came   Jill
Jack   came   hill

pail   did   Jill
pail   did   hill
pail   did   pail
pail   did   water
pail   did   crown
pail   did   Up
pail   did   home
pail   did   As
pail   did   head
pail   did   vinegar
pail   did   paper
pail   caper   Jack
pail   caper   Jill
pail   caper   hill
pail   caper   pail
pail   caper   water
pail   caper   crown
pail   caper   Up
pail   caper   home
pail   caper   As
pail   caper   head
pail   caper   vinegar
pail   caper   paper
pail   bed   Jack
pail   bed   Jill
pail   bed   hill
pail   bed   pail
pail   bed   water
pail   bed   crown
pail   bed   Up
pail   bed   home
pail   bed   As
pail   bed   head
pail   bed   vinegar
pail   bed   paper
pail   mend   Jack
pail   mend   Jill
pail   mend   hill
pail   mend   pail
pail   mend   water
pail   mend   crown
pail   mend   Up
pail   mend   home
pail   mend   As
pail   mend   head
pail   mend   vinegar
pail   mend   paper
water   went   Jack
water   went   Jill
water   went   hill
water   went   pail
water   went   water
water   w

vinegar   tumbling   Jill
vinegar   tumbling   hill
vinegar   tumbling   pail
vinegar   tumbling   water
vinegar   tumbling   crown
vinegar   tumbling   Up
vinegar   tumbling   home
vinegar   tumbling   As
vinegar   tumbling   head
vinegar   tumbling   vinegar
vinegar   tumbling   paper
vinegar   got   Jack
vinegar   got   Jill
vinegar   got   hill
vinegar   got   pail
vinegar   got   water
vinegar   got   crown
vinegar   got   Up
vinegar   got   home
vinegar   got   As
vinegar   got   head
vinegar   got   vinegar
vinegar   got   paper
vinegar   did   Jack
vinegar   did   Jill
vinegar   did   hill
vinegar   did   pail
vinegar   did   water
vinegar   did   crown
vinegar   did   Up
vinegar   did   home
vinegar   did   As
vinegar   did   head
vinegar   did   vinegar
vinegar   did   paper
vinegar   caper   Jack
vinegar   caper   Jill
vinegar   caper   hill
vinegar   caper   pail
vinegar   caper   water
vinegar   caper   crown
vinegar   caper   Up
vinegar   caper   home
vinegar   caper   As

To represent the words as unqiue vectors in a quantum state-space, we define each vector as a unique qubit index, wherein a two-way mapping exists between the word and the index. This simplifies life for us later, as we map the data into the quantum state-space, and figure out the result of the computation upon completion.

In [12]:
# Maps the unique string in each respective space to a binary basis number for qubit representation
def mapNameToBinaryBasis(words):
    verbMap = {}
    nounMap = {}
    for i,v in enumerate(words['nouns']):
        #Perform two-way mapping for bin to value and value to bin
        nounMap.update({v:format(i,'b').zfill(num_q_n)})
        nounMap.update({format(i,'b').zfill(num_q_n):v})

    for i,v in enumerate(words['verbs']):
        verbMap.update({v:format(i,'b').zfill(num_q_v)})
        verbMap.update({format(i,'b').zfill(num_q_v):v})
        
    return (nounMap, verbMap)

In [13]:
nMap,vMap = mapNameToBinaryBasis(words)

In [17]:
nMap

{'Jack': '0000',
 '0000': 'Jack',
 'Jill': '0001',
 '0001': 'Jill',
 'hill': '0010',
 '0010': 'hill',
 'pail': '0011',
 '0011': 'pail',
 'water': '0100',
 '0100': 'water',
 'crown': '0101',
 '0101': 'crown',
 'Up': '0110',
 '0110': 'Up',
 'home': '0111',
 '0111': 'home',
 'As': '1000',
 '1000': 'As',
 'head': '1001',
 '1001': 'head',
 'vinegar': '1010',
 '1010': 'vinegar',
 'paper': '1011',
 '1011': 'paper'}

In [21]:
vMap

{'went': '0000',
 '0000': 'went',
 'fetch': '0001',
 '0001': 'fetch',
 'fell': '0010',
 '0010': 'fell',
 'broke': '0011',
 '0011': 'broke',
 'came': '0100',
 '0100': 'came',
 'tumbling': '0101',
 '0101': 'tumbling',
 'got': '0110',
 '0110': 'got',
 'did': '0111',
 '0111': 'did',
 'caper': '1000',
 '1000': 'caper',
 'bed': '1001',
 '1001': 'bed',
 'mend': '1010',
 '1010': 'mend'}

# Prepare quantum states

In [19]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, \
        execute, QuantumCircuit, execute, Aer, IBMQ, compile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import quantum_info as qi
from qiskit import BasicAer as ba

In [15]:
#Create the required circuit to represent the data
qreg_n0 = QuantumRegister(num_q_n);
qreg_v = QuantumRegister(num_q_v);
qreg_n1 = QuantumRegister(num_q_n);

# $|1\rangle$ measurement indicates a value has been observed (true), 
# and a $|0\rangle$ indicates no observation (false)
qreg_TF = QuantumRegister(1);

#Results register for measuring the final states
c_q_n0 = ClassicalRegister(num_q_n);
c_q_v = ClassicalRegister(num_q_v); 
c_q_n1 = ClassicalRegister(num_q_n); 

c_q_TF = ClassicalRegister(1); 

circ = QuantumCircuit(qreg_n0, qreg_v, qreg_n1, c_q_n0, c_q_v, c_q_n1, qreg_TF, c_q_TF);

circ0 = QuantumCircuit(qreg_n0);
circ1 = QuantumCircuit(qreg_n1);

In [16]:
circ.draw()

To create the appropriate states, we must define the required superposition states necessary to represent the meaning. Mapping the binary strings to superposition states via operations will be the next step.

As a quick aside, let us examine the case of:
$$
\begin{pmatrix}
\textrm{Bill} \\
\textrm{Mark}
\end{pmatrix}
\otimes
\begin{pmatrix}
\textrm{loves} \\
\textrm{hates}
\end{pmatrix}
\otimes
\begin{pmatrix}
\textrm{cake} \\
\textrm{fish}
\end{pmatrix}
%\otimes
%\begin{pmatrix}
%\textrm{True} \\
%\textrm{False}
%\end{pmatrix}
$$

We wish to examine and test the sentences: "Bill hates fish, but Mark likes fish". The state that would encode the above information, assuming binary indexing for each qubit is: $\vert \Psi \rangle = (1/d)*(\vert0\rangle\vert1\rangle\vert1\rangle + \vert1\rangle\vert0\rangle\vert1\rangle)$, where $d$ provides normalisation. In this instance we may pose a question: "Does Bill love fish?". While we can achieve a result with this, it can be noted that the resulting overlap (upon simplification to conj(Bill loves fish)) gives us a scalar value immediately. Instead, what if we want to return the result in a state itself? In this instance it can be useful to include an additional qubit to track the result of the outcome, such as $|0 \rangle$ for false, and $|1\rangle$ for true (or somewhere in between for a more nuanced result). Our tensor state-space now becomes:

$$
\begin{pmatrix}
\textrm{Bill} \\
\textrm{Mark}
\end{pmatrix}
\otimes
\begin{pmatrix}
\textrm{loves} \\
\textrm{hates}
\end{pmatrix}
\otimes
\begin{pmatrix}
\textrm{cake} \\
\textrm{fish}
\end{pmatrix}
\otimes
\begin{pmatrix}
\textrm{True} \\
\textrm{False}
\end{pmatrix}
$$

$\vert \Psi^{\prime} \rangle = (1/d)*(\vert0\rangle\vert1\rangle\vert1\rangle\vert1\rangle + \vert1\rangle\vert0\rangle\vert1\rangle\vert1\rangle)$. As such, we may perform a projective measurement on first 3 qubits of our resulting state, and be left with a single qubit encoding the result of our question ('Bill loves fish == True' could be the interpretation in this instance). 

As this will destroy the superposition we began with, if we have a method to retrieve the required data from RAM and encode it into a readable state, it may allow us to pose repeated questions. Additionally, methods exist for returning the overlap of a given question with the entire register, returning the largest overlapping state to our problem.

In [ ]:
num_samples = 1000

circ.measure(qreg_n0, c_q_n0)
circ.measure(qreg_v, c_q_v)
circ.measure(qreg_n1, c_q_n1)

backend = Aer.get_backend('qasm_simulator')
qobj = compile(circ, backend, shots=num_samples)
job = backend.run(qobj)
result = job.result()
counts = result.get_counts()
print(counts)

In [ ]:
#Demonstration of fidelity calculation from state-vector values
test_vec = [0.5]*4
qr = QuantumRegister(2, "qr")
qc = QuantumCircuit(qr)
qc.initialize(test_vec, [qr[0], qr[1]])
job = execute(qc, ba.get_backend('statevector_simulator'))
result = job.result()
statevector = result.get_statevector()
fidelity = state_fidelity(statevector, test_vec)

print(fidelity)

In [ ]:
circ0.z(qreg_n0[0])
circ0.x(qreg_n0[1])
job0 = execute(circ0, BasicAer.get_backend('statevector_simulator'))

circ1.x(qreg_n1[0])
circ1.z(qreg_n1[1])
job1 = execute(circ1, BasicAer.get_backend('statevector_simulator'))

In [ ]:
result0 = job0.result()
result1 = job1.result()
statevector0 = result0.get_statevector()
statevector1 = result1.get_statevector()

In [ ]:
print(statevector0)
print(statevector1)
np.abs(np.dot(np.conj(statevector1),statevector0))**2

In [ ]:
qi.state_fidelity(statevector0, statevector1)